In [ ]:
import pandas as pd
import os
import zipfile
import numpy as np
import gc
import pickle
import warnings
import tensorflow as tf
import tensorflow_model_optimization as tfmot

warnings.filterwarnings('ignore')

Import dataset pickle file

In [ ]:
def get_pickle(pickle_dir, file_name):
    pickle_dir = os.path.join(os.getcwd(), 'datasets/fsc22/Pickle Files/' + pickle_dir)
    fold_dir = os.path.join(pickle_dir, file_name)
    infile = open(fold_dir, 'rb')
    fold = pickle.load(infile)
    infile.close()

    return fold

In [ ]:
with tf.device('/CPU:0'):
    pickle_dir = 'aug_ts_ps_mixed_features_5_20_5fold' # change pickle file directory name accordingly
    
    spect_folds = []
    train_spects = []
    
    for fold in range(5):
        spects_fold = get_pickle(pickle_dir, pickle_dir + '_fold' + str(fold+1))
        train_spects.extend(spects_fold)
        spect_folds.append(spects_fold)

    print(f'len train_spects: {len(train_spects)}')

    train_features_df = pd.DataFrame(train_spects, columns=['feature', 'class'])

    del train_spects

    gc.collect()
    
    print(f'len spect_folds: {len(spect_folds)}') # num folds
    print(f'len spect_folds[0]: {len(spect_folds[0])}') # samples in a fold
    print(f'len spect_folds[0][0]: {len(spect_folds[0][0])}') # elements in a sample - should be 2
    print(f'shape spect_folds[0][0][0]: {np.shape(spect_folds[0][0][0])}') # spectrogram shape

In [ ]:
with tf.device('/CPU:0'):
    IMG_SIZE = (spect_folds[0][0][0].shape[0], spect_folds[0][0][0].shape[1])
    IMG_SHAPE = IMG_SIZE + (3,)
    print(IMG_SHAPE)
    input_shape = IMG_SHAPE

    num_labels = 26 # Should be changed

Import base model

In [ ]:
with tf.device('/CPU:0'):
    valid_model_type = False
    model_type = None

    while not valid_model_type:
        model_type = input(
            "Choose Model Type:\n"
            " 1. AlexNet\n"
            " 2. DenseNet121\n"
            " 3. EfficientNetV2B0\n"
            " 4. InceptionV3\n"
            " 5. MobileNetV3Small\n"
            " 6. ResNet50V2\n"
            " 7. SqueezeNet\n :")

        if model_type == '1':
            model_type = 'AlexNet'
            valid_model_type = True
        elif model_type =='2':
            model_type = 'DenseNet121'
            valid_model_type = True
        elif model_type =='3':
            model_type = 'EfficientNetV2B0'
            valid_model_type = True
        elif model_type =='4':
            model_type = 'InceptionV3'
            valid_model_type = True
        elif model_type =='5':
            model_type = 'MobileNetV3-Small'
            valid_model_type = True
        elif model_type =='6':
            model_type = 'ResNet50V2'
            valid_model_type = True
        elif model_type =='7':
            model_type = 'SqueezeNet'
            valid_model_type = True

In [ ]:
with tf.device('/CPU:0'):
    models_dir = os.path.join(os.getcwd(), "models")
    model_dir = os.path.join(models_dir, model_type)

    # Example - 
    # model_name = 'DenseNet_aug5_mix_5fold'
    # base_model_save_path = 'models/DenseNet121/DenseNet_aug5_mix_5fold/DenseNet_aug5_mix_5fold_base_fold1.h5'
    # fold_no = 1
    
    model_name = input('Enter model name: ')
    base_model_save_path = input('Enter base model relative save path: ')
    fold_no = int(input('Enter the fold which the base model was validated: '))

    model_save_dir = os.path.join(model_dir, model_name)
    
    base_model = tf.keras.models.load_model(base_model_save_path)
    base_model.summary()

Create train and validation data

In [ ]:
def get_train_vaild_data(spects, fold):
    train_spects = []
    valid_spects = None
    
    for i in range(5):
        if i + 1 != fold:
            train_spects.extend(spects[i])
        else:
            valid_spects = spects[i]
    
    train_df = pd.DataFrame(train_spects, columns=['feature', 'class'])
    valid_df = pd.DataFrame(valid_spects, columns=['feature', 'class'])

    del train_spects
    del valid_spects

    gc.collect()

    X_train_cv = np.array(train_df['feature'].tolist())
    y_train_cv = np.array(train_df['class'].tolist())
    
    X_valid_cv = np.array(valid_df['feature'].tolist())
    y_valid_cv = np.array(valid_df['class'].tolist())
    
    return X_train_cv, X_valid_cv, y_train_cv, y_valid_cv

In [ ]:
X_train_cv, X_valid_cv, y_train_cv, y_valid_cv = get_train_vaild_data(spect_folds, fold_no)

print(f'X_train_cv shape: {np.shape(X_train_cv)}')
print(f'y_train_cv shape: {np.shape(y_train_cv)}')
print(f'X_valid_cv shape: {np.shape(X_valid_cv)}')
print(f'y_valid_cv shape: {np.shape(y_valid_cv)}')

Create base model

In [ ]:
# Change according to the hyperparameters of the base model

lr = 0.07619493236
best_optimizer = 'sgd'
batch_size = 32
epochs = 5

optimizer = tf.keras.optimizers.Adam(learning_rate=lr) if best_optimizer=='adam' else tf.keras.optimizers.SGD(learning_rate=lr)
base_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

base_model_loss, base_model_accuracy = base_model.evaluate(x=X_valid_cv,y=y_valid_cv)

### Weight pruning

Define weight pruning schedule and affected layers

In [ ]:
# Helper function uses `prune_low_magnitude` to make only the Selected layers train with pruning.
num_images = X_train_cv.shape[0]
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

final_sparsity = float(input('Enter final sparsity: ')) # Example - 0.7

def apply_pruning_to_layer(layer):
    pruning_params = {
          'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.0,
                                                                final_sparsity=final_sparsity,
                                                                begin_step=0,
                                                                end_step=end_step)
    }

    if isinstance(layer, tf.keras.layers.Dense):
        return tfmot.sparsity.keras.prune_low_magnitude(layer, **pruning_params)
    elif isinstance(layer, tf.keras.layers.Conv2D):
        return tfmot.sparsity.keras.prune_low_magnitude(layer, **pruning_params)
    elif isinstance(layer, tf.keras.layers.BatchNormalization):
        return tfmot.sparsity.keras.prune_low_magnitude(layer, **pruning_params)

    return layer

Apply weight pruning mask to model

In [ ]:
# Use `tf.keras.models.clone_model` to apply `apply_pruning_to_dense`
# to the layers of the model.
model_for_pruning = tf.keras.models.clone_model(
    base_model,
    clone_function=apply_pruning_to_layer,
)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(
      optimizer=optimizer,
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy']
      )

model_for_pruning.summary()

Filter prune model by training it with weight pruning callback

In [ ]:
with tf.device('/CPU:0'):
    logdir = os.path.join(model_save_dir, 'pruning_summaries')

    if not os.path.exists(logdir):
        os.makedirs(logdir)

    callbacks = [
        tfmot.sparsity.keras.UpdatePruningStep(),
        tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
    ]

    model_for_pruning.fit(X_train_cv, y_train_cv,
                        batch_size=batch_size, epochs=epochs, validation_data=(X_valid_cv, y_valid_cv),
                        callbacks=callbacks)

In [ ]:
pruned_model_loss, pruned_model_accuracy = model_for_pruning.evaluate(x=X_valid_cv,y=y_valid_cv)

print('Baseline test accuracy:', base_model_accuracy) 
print('Pruned test accuracy:', pruned_model_accuracy)

### Save weight pruned model

In [ ]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

pruned_model_name = model_name + "_weight_pruned_" + str(final_sparsity) + "_fold" + str(fold_no)

weight_pruned_model_save_path = os.path.join(model_save_dir, pruned_model_name + ".h5")

tf.keras.models.save_model(model_for_export, weight_pruned_model_save_path, include_optimizer=False)

Save weight pruned model as a Tensorflow Lite model

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

weight_pruned_tflite_model_save_path = os.path.join(model_save_dir, pruned_model_name + ".tflite")

with open(weight_pruned_tflite_model_save_path, 'wb') as f:
    f.write(pruned_tflite_model)

In [ ]:
def get_gzipped_model_size(file, model_type):
    # Returns size of gzipped model, in bytes.

    zipped_file = os.path.join(model_save_dir, model_name + "_" + model_type + ".zip")
    with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
        f.write(file)

    return os.path.getsize(zipped_file)

In [ ]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(base_model_save_path, 'base')))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(weight_pruned_model_save_path, 'weight_pruned')))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(weight_pruned_tflite_model_save_path, '_weight_pruned_tflite')))